In [1]:
import pandas as pd

In [2]:
campione = pd.read_csv("./campione_1500.csv")
print("Numero Coppie nel Campione:", len(campione))
print("Numero Coppie 'Matching' nel Campione:", len(campione[campione['Label'] == 1]))
print("Numero Coppie 'Non Matching' nel Campione:", len(campione[campione['Label'] == 0]))

Numero Coppie nel Campione: 1538
Numero Coppie 'Matching' nel Campione: 773
Numero Coppie 'Non Matching' nel Campione: 765


# Valutazione del Blocking con Word2Vec
Calcoleremo Precision e Recall utilizzando il seguente formalismo:
- True Negative -> Coppie 'Non Matching' che non finiscono nello stesso Blocco
- False Positive -> Coppie 'Non Matching' che finiscono nello stesso Blocco
- False Negative -> Coppie 'Matching' che non finiscono nello stesso Blocco
- True Positive -> Coppie 'Matching' che finiscono nello stesso Blocco

In [3]:
schema_mediato_con_blocks = pd.read_csv("./../datasets/MediatedSchemaConBlocks_W2V.csv")

In [4]:
tn, fp, fn, tp = list(), list(), list(), list()

for i in range(0, len(campione)):
    row = campione.iloc[i]
    first, second, true_label = row['Riga_SX'], row['Riga_DX'], row['Label']

    first_block = schema_mediato_con_blocks.iloc[first]['block']
    second_block = schema_mediato_con_blocks.iloc[second]['block']

    computed_label = None
    if first_block == second_block:
        computed_label = 1
    else:
        computed_label = 0
    
    if true_label == 0 and computed_label == 0:
        tn.append((first, second))
    if true_label == 0 and computed_label == 1:
        fp.append((first, second))
    if true_label == 1 and computed_label == 0:
        fn.append((first, second))
    if true_label == 1 and computed_label == 1:
        tp.append((first, second))

In [5]:
recall = len(tp) / (len(tp) + len(fn))
precision = len(tp) / (len(tp) + len(fp))
f_measure = (2 * precision * recall) / (precision + recall)

print("Recall:", recall)
print("Precision:", precision)
print("F-Measure:", f_measure)

Recall: 0.4191461836998706
Precision: 0.8828337874659401
F-Measure: 0.5684210526315789


# Valutazione del Pairwise Matching sul Blocking con Word2Vec

In [6]:
import re

In [7]:
# Apre il file in modalità di lettura
with open('./../3 - Pairwise Matching/results_clustersW2V.txt', 'r') as file:
    # Legge le righe del file
    lines = file.readlines()

# Inizializza un dizionario vuoto
result_dict = {}

# Itera su ogni riga del file
for line in lines:
    match = re.match(r"(\d+):\s*({.*})\s*", line)
    if match:
        key = int(match.group(1))
        value = match.group(2)
        value = eval(value)
        result_dict[key] = value

In [8]:
tn, fp, fn, tp = 0, 0, 0, 0
for i in range(0, len(campione)):
    row = campione.iloc[i]
    first, second, label = row['Riga_SX'], row['Riga_DX'], row['Label']

    if label == 0:
        first_b, second_b, first_cl, second_cl = None, None, None, None
        for key_i in list(result_dict.keys()):
            inner_dict = result_dict[key_i]
            for key_j in list(inner_dict.keys()):
                if first in inner_dict[key_j]:
                    first_b, first_cl = key_i, key_j
                if second in inner_dict[key_j]:
                    second_b, second_cl = key_i, key_j
        
        if first_b != second_b or (first_b == second_b and first_cl != second_cl):
            tn += 1
        if first_b == second_b and first_cl == second_cl:
            fp += 1

    elif label == 1:
        first_b, second_b, first_cl, second_cl = None, None, None, None
        for key_i in list(result_dict.keys()):
            inner_dict = result_dict[key_i]
            for key_j in list(inner_dict.keys()):
                if first in inner_dict[key_j]:
                    first_b, first_cl = key_i, key_j
                if second in inner_dict[key_j]:
                    second_b, second_cl = key_i, key_j
        
        if first_b != second_b or (first_b == second_b and first_cl != second_cl):
            fn += 1
        if first_b == second_b and first_cl == second_cl:
            tp += 1

In [9]:
recall = tp / (tp + fn)
precision = tp / (tp + fp)
f_measure = 2*recall*precision / (recall+precision)

print("Recall:", recall)
print("Precision:", precision)
print("F-Measure:", f_measure)

Recall: 0.1423027166882277
Precision: 0.9821428571428571
F-Measure: 0.24858757062146897


In [12]:
record_rimanenti = 0
for key_i in list(result_dict.keys()):
    inner_dict = result_dict[key_i]
    record_rimanenti += len(inner_dict)

print("Record rimanenti dopo il Record Linkage:", record_rimanenti)
print("Percentuale 'sopravvissuta' del Dataset iniziale:", float(100*record_rimanenti/len(schema_mediato_con_blocks)))

Record rimanenti dopo il Record Linkage: 69122
Percentuale 'sopravvissuta' del Dataset iniziale: 84.5984383031846
